__Defining model VGG3 with dropout__

In [49]:
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import RMSprop
from keras import layers
import tensorflow as tf


def define_model():
	model = Sequential([
    layers.Conv2D(16, (3,3), activation='selu', input_shape=(200,200,3)),
    layers.MaxPool2D(2,2),
    layers.Conv2D(32, (3,3), activation='selu'),
    layers.MaxPool2D(2,2),
    layers.Dropout(0.05),
     
    layers.Conv2D(64, (3,3), activation='selu'),
    layers.MaxPool2D(2,2),
    layers.Dropout(0.1),
	
    layers.Conv2D(128, (2,2), activation='selu'),
    layers.MaxPool2D(2,2),
    layers.Conv2D(256, (2,2), activation='selu'),
    layers.MaxPool2D(2,2),
    layers.Dropout(0.2),
	
    layers.Flatten(),
    layers.Dense(500, activation='selu'),
 
    layers.Dense(1, activation='sigmoid')                        
])
	model.compile(loss='binary_crossentropy',
            optimizer = RMSprop(learning_rate=0.001),
            #optimizer=tf.keras.optimizers.Adam(learning_rate=0.000244),
             metrics = ['binary_accuracy'])
	model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
	filepath = "model_checkp.keras",
    monitor='val_binary_accuracy',
    mode='max',
    save_best_only=True)
	return model

model = define_model()

__Creating data-generator__

to rescale and categorize images using shifts and horizontal flip 

In [50]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Создаем DataGen который будет каждый пиксель нормализовать от 0 до 1
datagen = ImageDataGenerator(rescale=1.0/255.0, 
                             width_shift_range=0.2, height_shift_range=0.2, 
                             horizontal_flip=True)

train_gen = datagen.flow_from_directory(
    'train/',
    target_size=(200, 200),
    batch_size=64,
    class_mode='binary')
test_gen  = datagen.flow_from_directory(
    'test/',
	class_mode='binary', 
    batch_size=64,
    target_size=(200, 200))

Found 19911 images belonging to 2 classes.
Found 5089 images belonging to 2 classes.


__Fitting model and checking acc__

In [52]:
history = model.fit(train_gen, steps_per_epoch=len(train_gen),
	validation_data=test_gen, validation_steps=len(test_gen), epochs=20, verbose=1)
_, acc = model.evaluate(test_gen, steps=len(test_gen), verbose=0)
'> %.3f' % (acc * 100.0)
#3 эпохи 50%

Epoch 1/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 170s 539ms/step - binary_accuracy: 0.6279 - loss: 0.6661 - val_binary_accuracy: 0.6826 - val_loss: 0.6054
Epoch 2/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 37us/step - binary_accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 169s 535ms/step - binary_accuracy: 0.6245 - loss: 0.7585 - val_binary_accuracy: 0.6885 - val_loss: 0.5782
Epoch 4/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 47us/step - binary_accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 169s 537ms/step - binary_accuracy: 0.6653 - loss: 0.6180 - val_binary_accuracy: 0.5769 - val_loss: 0.7706
Epoch 6/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 41us/step - binary_accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 168s 534ms/step - binary_accuracy: 0.6915 - l

'> 81.332'

In [53]:
model.save('cat_or_dog_img_classifier.h5')